In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from dask import compute, delayed
from time import perf_counter

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 1

In [4]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [6]:
print(survival_df.shape)
survival_df.head()

(46, 13)


,sample_name,survival_time,vital_status,figo_stage,days_to_last_follow_up,days_to_death,age_at_diagnosis,age_at_index,height,weight,bmi,race,ethnicity
0,TCGA-N5-A4R8-01A-11R-A28V-07,3115.0,1,Stage IA,NaN,3115.0,24102.0,65.0,164.0,54.0,20.077335,white,not hispanic or latino
1,TCGA-N5-A4RA-01A-11R-A28V-07,1591.0,1,Stage IIIC2,NaN,1591.0,23359.0,63.0,152.0,72.0,31.163435,white,hispanic or latino
2,TCGA-N8-A4PM-01A-11R-A28V-07,829.0,0,Stage IB,829.0,NaN,26971.0,73.0,155.0,92.0,38.293444,white,not hispanic or latino
3,TCGA-NA-A4QW-01A-11R-A28V-07,2457.0,0,Stage IIA,2457.0,NaN,22588.0,61.0,155.0,66.0,27.471384,white,not hispanic or latino
4,TCGA-NG-A4VW-01A-11R-A28V-07,949.0,1,Stage IC,NaN,949.0,27326.0,74.0,170.0,81.0,28.027682,white,not hispanic or latino


In [7]:
filtered_survival_df = (
    survival_df[["sample_name"] + dep_cols]
        .query("vital_status == 1")
        .dropna()
        .reset_index(drop=True)
)
print(filtered_survival_df.shape)
print(filtered_survival_df.shape[0] / survival_df.shape[0])
filtered_survival_df.head()

(27, 3)
0.5869565217391305


,sample_name,vital_status,survival_time
0,TCGA-N5-A4R8-01A-11R-A28V-07,1,3115.0
1,TCGA-N5-A4RA-01A-11R-A28V-07,1,1591.0
2,TCGA-NG-A4VW-01A-11R-A28V-07,1,949.0
3,TCGA-N8-A4PI-01A-21R-A28V-07,1,810.0
4,TCGA-NA-A4R0-01A-11R-A28V-07,1,541.0


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .drop("vital_status", axis=1)
        .set_index("sample_name")
)

print(joined_df.shape)
joined_df.head()

(27, 1009)


,survival_time,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,NDNF,...,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
sample_name,,,,,,,,,,,,,,,,,,,,,
TCGA-N5-A4R8-01A-11R-A28V-07,3115.0,11.071504,6.193957,9.188275,11.935509,7.162635,12.897590,4.804702,4.677573,6.916997,...,6.504208,8.179961,9.125170,9.282168,9.350748,4.804702,4.235667,14.177946,9.701065,10.871553
TCGA-N5-A4RA-01A-11R-A28V-07,1591.0,13.337561,8.812780,8.815553,13.025127,11.745715,7.978252,4.235667,4.235667,5.599775,...,6.216415,8.215729,7.605536,9.425110,9.524951,5.147093,4.919925,11.383441,8.999895,13.981692
TCGA-NG-A4VW-01A-11R-A28V-07,949.0,8.212508,5.010694,10.197487,13.866272,4.531615,10.110854,4.824506,4.235667,11.248957,...,8.940214,6.325351,8.421775,12.323383,13.057145,9.178995,4.235667,9.489794,6.813619,13.973405
TCGA-N8-A4PI-01A-21R-A28V-07,810.0,9.085991,6.089452,9.505489,13.864509,4.762966,5.273615,5.062679,4.235667,8.447244,...,8.349010,5.518576,9.077512,13.202840,6.303639,6.982836,4.235667,12.858102,5.209084,14.087779
TCGA-NA-A4R0-01A-11R-A28V-07,541.0,9.300812,7.553015,11.863714,11.682761,6.254591,8.493726,5.680264,4.235667,8.207741,...,10.842729,6.941994,9.345358,13.459929,7.403850,7.947018,4.235667,12.796196,6.658910,13.433016


# Examine mutual information

In [10]:
X = joined_df.iloc[:, 1:].values
y = joined_df.iloc[:, 0].values

In [11]:
rand.seed(seed)
sim_rounds = 101
start = perf_counter()
mi_delayed = [delayed(mutual_info_regression)(X, y, discrete_features=False, random_state=rand) for _ in range(sim_rounds)]
res = compute(*mi_delayed, scheduler="processes")
stop = perf_counter()
print(stop - start)

22.230509999964852


In [12]:
mi_df = pd.concat([
    pd.DataFrame({"geneID": joined_df.columns[1:]}),
    pd.DataFrame(np.column_stack(res), columns=[f"MI_est_{i + 1}" for i in range(sim_rounds)])
], axis=1)
mi_df["MI_est_median"] = mi_df.iloc[:, 1:].median(axis=1)
mi_df.head()

,geneID,MI_est_1,MI_est_2,MI_est_3,MI_est_4,MI_est_5,MI_est_6,MI_est_7,MI_est_8,MI_est_9,...,MI_est_93,MI_est_94,MI_est_95,MI_est_96,MI_est_97,MI_est_98,MI_est_99,MI_est_100,MI_est_101,MI_est_median
0,PGF,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,TIMP4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,C1QTNF6,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,TNC,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,...,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596,0.00596
4,PRL,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [13]:
sum(mi_df.MI_est_median > 0)

447

In [14]:
mi_df[["geneID", "MI_est_median"]].to_csv(f"{dirs.analysis_dir}/{unified_dsets[dset_idx]}_MI_survival_results.tsv", sep="\t", index=False)